<a href="https://colab.research.google.com/github/lorenzoR21/Computer-Vision-Project/blob/main/notebook/parking_slot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-time Parking Slot Occupancy Detection

---

## 📘 Overview
This is a supporting secondary notebook, which contains the definition and training of a model that extracts the coordinates of individual parking slots from entire images of parking spaces. The pre-trained FasterRCNN-FPN model from pytorch is used as the model.

---

## 📝 Authors
- **Lorenzo Russo**  
  Email: russo.2091186@studenti.uniroma1.it

---

## 🔗 Useful Links
- [Project Repository](https://github.com/lorenzoR21/Computer-Vision-Project)

---




Install all needed library

In [ ]:
!pip install pytorch_lightning

Import all needed library

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import Accuracy, AUROC
from torch.utils.data import DataLoader, Dataset, ConcatDataset, random_split
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

## Dataset

In [ ]:
!wget "https://github.com/fabiocarrara/deep-parking/releases/download/archive/CNR-EXT_FULL_IMAGE_1000x750.tar" -O "CNRPark_FULL_IMAGE.tar"
!mkdir CNR-EXT_FULL_IMAGE_1000x750
!tar -xf CNRPark_FULL_IMAGE.tar -C CNR-EXT_FULL_IMAGE_1000x750 && rm CNRPark_FULL_IMAGE.tar

In [ ]:
class EntireParkingDataset(Dataset):
    def __init__(self, root_dir, annotation_dir, transform=None):
        self.root_dir = root_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.image_files = self._get_image_files()
        self.annotations = self._load_annotations()

    def _get_image_files(self):
        image_files = []
        for date_dir in os.listdir(self.root_dir):
            date_path = os.path.join(self.root_dir, date_dir)
            if os.path.isdir(date_path):
                for camera_dir in os.listdir(date_path):
                    camera_path = os.path.join(date_path, camera_dir)
                    if os.path.isdir(camera_path):
                        for img_file in os.listdir(camera_path):
                            if img_file.endswith('.jpg'):
                                image_files.append((int(camera_path[-1]), os.path.join(camera_path, img_file)))
        return image_files

    def _load_annotations(self):
        annotations = {}
        scale_factor_x = 1000 / 2592
        scale_factor_y = 750 / 1944
        for camera in range(1, 10):
            annot_path = os.path.join(self.annotation_dir, f'camera{camera}.csv')
            df = pd.read_csv(annot_path)
            for _, row in df.iterrows():
                slot_id = row['SlotId']
                x_large, y_large, w_large, h_large = int(row['X']), int(row['Y']), int(row['W']), int(row['H'])

                x_small = int(x_large * scale_factor_x)
                y_small = int(y_large * scale_factor_y)
                w_small = int(w_large * scale_factor_x)
                h_small = int(h_large * scale_factor_y)

                left = x_small
                upper = y_small
                right = x_small + w_small
                lower = y_small + h_small

                if camera not in annotations:
                    annotations[camera] = []
                annotations[camera].append([left, upper, right, lower])
        return annotations

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        camera, img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        b = self.annotations.get(camera, [])

        boxes = []
        labels = []
        for slot in b:
            x1, y1, x2, y2 = slot
            boxes.append([x1, y1, x2, y2])
            labels.append(1)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels
        }

        if self.transform:
            image, target = self.transform(image, target)

        return image, target

In [ ]:
class Resize:
    def __init__(self, size=(450, 600)):
        self.size = size
        self.transform = transforms.Compose([
            transforms.Resize(self.size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __call__(self, image, target=None):
        w, h = image.size
        image = self.transform(image)
        new_h, new_w = self.size

        if target:
            boxes = target['boxes']
            boxes[:, 0] = boxes[:, 0] * (new_w / w)
            boxes[:, 1] = boxes[:, 1] * (new_h / h)
            boxes[:, 2] = boxes[:, 2] * (new_w / w)
            boxes[:, 3] = boxes[:, 3] * (new_h / h)
            target['boxes'] = boxes

        return image, target

def collate_fn(batch):
    return tuple(zip(*batch))

transform = Resize()

In [ ]:
dataset_sunny = EntireParkingDataset(root_dir='CNR-EXT_FULL_IMAGE_1000x750/FULL_IMAGE_1000x750/SUNNY', annotation_dir='CNR-EXT_FULL_IMAGE_1000x750', transform=transform)
dataset_rainy = EntireParkingDataset(root_dir='CNR-EXT_FULL_IMAGE_1000x750/FULL_IMAGE_1000x750/RAINY', annotation_dir='CNR-EXT_FULL_IMAGE_1000x750', transform=transform)
dataset_overcast = EntireParkingDataset(root_dir='CNR-EXT_FULL_IMAGE_1000x750/FULL_IMAGE_1000x750/OVERCAST', annotation_dir='CNR-EXT_FULL_IMAGE_1000x750', transform=transform)
dataset = ConcatDataset([dataset_sunny, dataset_rainy, dataset_overcast])

total_length = len(dataset)
train_length = int(0.7 * total_length)
val_length = int(0.1 * total_length)
test_length = total_length - train_length - val_length

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_length, val_length, test_length])

train_dataset.transforms = transform
val_dataset.transforms = transform
test_dataset.transforms = transform

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, collate_fn=collate_fn)

## Extract Parking Slot with FasterRCNN_FPN

In [ ]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
class ParkingSlotDetectionModel(pl.LightningModule):
    def __init__(self, num_classes):
        super(ParkingSlotDetectionModel, self).__init__()
        self.model = get_model(num_classes)

    def forward(self, images, targets=None):
        if targets:
            output = self.model(images, targets)
            return output
        return self.model(images)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]
        loss_dict = self.forward(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        self.log('train_loss', losses)
        return losses

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]
        self.model.train()
        with torch.no_grad():
            loss_dict = self.forward(images, targets)
        self.model.eval()
        losses = sum(loss for loss in loss_dict.values())
        self.log('val_loss', losses)
        return losses

    def test_step(self, batch, batch_idx):
        images, targets = batch
        images = list(image for image in images)
        targets = [{k: v for k, v in t.items()} for t in targets]
        self.model.train()
        with torch.no_grad():
            loss_dict = self.forward(images, targets)
        self.model.eval()
        losses = sum(loss for loss in loss_dict.values())
        self.log('test_loss', losses)
        return losses

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
        return [optimizer], [lr_scheduler]

In [ ]:
tb_logger = TensorBoardLogger('logs/', name='parking_slot_detection')

In [ ]:
model = ParkingSlotDetectionModel(num_classes=2)
trainer = Trainer(max_epochs=10, accelerator='gpu', devices=1 if torch.cuda.is_available() else 0, logger=tb_logger)

Train the model

In [ ]:
trainer.fit(model, train_loader, val_loader)

Test the model

In [ ]:
trainer.test(model, test_loader)

Save the trained model

In [ ]:
torch.save(model.state_dict(), 'parking_slot_detection.pth')

Analyze loss and metrics

In [ ]:
%load_ext tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs/